In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")

from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

# 参数设置

# 训练参数 
除了路径 其他不要变

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
# TODO change the path as relative path
args.to_save_path     = r"E:\TECO_Works\Conference\ISWC2023\Run_logs"              
args.freq_save_path   = r"E:\TECO_Works\Conference\ISWC2023\Freq_data"
args.window_save_path = r"E:\TECO_Works\Conference\ISWC2023\Sliding_window"
args.root_path        = r"E:\datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 7
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

## 数据参数



In [3]:
args.seed                             = 1


args.data_name                        =  "hapt"

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 6


# 是否作augmentation difference
args.difference          =  True 

# 是否作augmentation  filtering
args.filtering           =  True

# 是否作augmentation  magnitude
args.magnitude           =  False
args.weighted_sampler    = False



# MIXUP
args.mixup_probability  = 0.5
args.mixup_alpha  = 0.5


# Random Augmentation
args.random_augmentation_prob = 0.5
args.random_augmentation_config = {"jitter":True,
                                   "moving_average":True,
                                   "magnitude_scaling":True,
                                   "magnitude_warp":True,
                                   "magnitude_shift":True,
                                   "time_warp":True,
                                   "window_warp":True,
                                   "window_slice":True,
                                   "random_sampling":True,
                                   "slope_adding":True
                                   }
random_augmentation_nr = 0
for key in args.random_augmentation_config.keys():
    if args.random_augmentation_config[key]:
        random_augmentation_nr = random_augmentation_nr+1
args.random_augmentation_nr = random_augmentation_nr
args.max_aug = 3

args.pos_select       = None
args.sensor_select    = None


args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
    
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            =  config["num_channels"]
print(args.c_in )
if args.difference:
    args.c_in  = args.c_in * 2
    print(args.c_in )
if  args.filtering :
    for col in config["sensors"]:
        if "acc" in col:
            args.c_in = args.c_in+1
            print(args.c_in )


if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1



6
12
13
14
15


In [4]:
args.c_in

15

## 模型参数

In [5]:
args.filter_scaling_factor = 0.25
args.model_type              = "deepconvlstm"#"deepconvlstm"#"sahar" #"deepconvlstm"  "deepconvlstm_attn"
exp = Exp(args)
exp.model

Use GPU: cuda:0


FileNotFoundError: [Errno 2] No such file or directory: 'configs/model.yaml'

In [6]:
exp.train()

 ----------------------- load all the data -------------------
Channel Augmentation : Differencing
Channel Augmentation : Acc Gyro Filtering
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 10 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.00066845 0.00067935 0.00073692 0.00060976 0.00055866 0.00056529
 0.00793651 0.01052632 0.00645161 0.00724638 0.00555556 0.00746269]
Train data number :  10352
The number of classes is :  12
The input_length  is :  128
The channel_in is :  15
Validation data number :  1151
Test data number :  6629
================ Build the model ================ 
Build the DeepConvLSTM model!
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128,

Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) t

Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size

torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]

VALI: Epoch: 9, Steps: 41 | Train Loss: 1.5365071  Vali Loss: 0.9026795 Vali Accuracy: 0.7871416  Vali weighted F1: 0.7525311  Vali macro F1 0.4202927 
new best score!!!!
Validation loss decreased (1.014263 --> 0.902680).  Saving model ...
new best score!!!!
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15

...

torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size

torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
Mixup torch.Size([256, 128, 15]) torch.Size([256, 3])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256])
outputs  torch.Size([256, 12])
torch.Size([256, 128, 15]) torch.Size([256]

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x0000029ED2AF7940> (for post_execute):



KeyboardInterrupt



In [5]:
# cross_channel_interaction_type = "attn",    # attn  transformer  identity
# cross_channel_aggregation_type = "filter",  # filter  naive  FC  "SFCC", "SFCF"
# temporal_info_interaction_type = "gru",     # gru  lstm  attn  transformer  identity  conv
# temporal_info_aggregation_type = "FC",      # naive  filter  FC  tnaive

# 实验

In [8]:
# 如果我们设置为 


args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"

exp = Exp(args)

Use GPU: cuda:0
Build the TinyHAR model!
Done!
Parameter : 370082
Set the seed as :  1


In [5]:
exp.model

model_builder(
  (model): MCNN(
    (conv_blocks): ModuleList(
      (0): ConvBlock(
        (conv1): Conv2d(1, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
      (1): ConvBlock(
        (conv1): Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
    )
    (dropout): Dropout(p=0.25, inplace=False)
    (activation): ReLU()
    (fc_sensor_fusion): Linear(in_features=48, out_features=32, bias=True)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc_temporal_fusion): Linear(in_features=608, out_features=32, bias=True)
    (fc_prediction): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [5]:
exp.train()

 ----------------------- load all the data -------------------
Channel Augmentation : Differencing
Channel Augmentation : Acc Gyro Filtering
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 10 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.00066845 0.00067935 0.00073692 0.00060976 0.00055866 0.00056529
 0.00793651 0.01052632 0.00645161 0.00724638 0.00555556 0.00746269]
Train data number :  10352
The number of classes is :  12
The input_length  is :  128
The channel_in is :  15
Validation data number :  1151
Test data number :  6629
================ Build the model ================ 
Build the DeepConvLSTM model!


ValueError: Expected input batch_size (128) to match target batch_size (256).

In [ ]:
dataset = data_dict[args.data_name](args)

In [ ]:
import numpy as np

In [ ]:
start_index = dataset.train_slidingwindows[0][1]
end_index   = dataset.train_slidingwindows[0][2]
sample_x_1    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

start_index = dataset.train_slidingwindows[100][1]
end_index   = dataset.train_slidingwindows[100][2]
sample_x_2    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

temp_1 = np.expand_dims(sample_x_1,0)
temp_2 = np.expand_dims(sample_x_2,0)
combined_x = np.concatenate([temp_1,temp_2],axis=0)
combined_x = np.expand_dims(combined_x,1)
combined_x = torch.tensor(combined_x).double().to(exp.device)
print(combined_x.shape)

In [ ]:
out = exp.model(combined_x)
out = out.cpu().detach().numpy()

In [ ]:
out.shape

In [ ]:
weight = exp.model.wave_conv.wavelet_conv.weight.cpu().detach().numpy()

In [ ]:
for n,p in exp.model.named_parameters():
    print(n)

In [ ]:
import matplotlib.pyplot as plt
index1 = 0
index2 = 0

plt.figure()
plt.plot(out[0,0,:,index2])
plt.plot(out[0,1,:,index2])
plt.plot(out[0,2,:,index2])
plt.plot(out[0,3,:,index2])

In [ ]:
# 如果我们设置为 
# args.wavelet_filtering_learnable      = True
# exp = Exp(args)